### If you want to use this code, you need to input it at the bottom of the pareto notebook.

## Peter's algo

In [ ]:
import math

def peter_algo():
        
    # Peter's load:
    peak_load = 35 * 1500 * 0.000293071
    load = [peak_load * (1 + 0.25 * math.cos(ix * (2 * math.pi)/24) - 0.25) for ix in range(24)]
    
    # Peter's heating capacity
    heating_capacity = [25.417404, 25.460465, 25.584543, 25.775322, 26.011996, 26.270950, 
            26.529450, 26.768482, 26.974246, 27.138260, 27.256380, 27.327268, 
            27.350861, 27.327268, 27.256380, 27.138260, 26.974246, 26.768482, 
            26.529450, 26.270950, 26.011996, 25.775322, 25.584543, 25.460465]
    
    # Peter's cost_th
    cost_th_list = [0.07919, 0.066283, 0.063061, 0.067943, 0.080084, 0.115845, 
                    0.193755, 0.215921, 0.110822, 0.044927, 0.01521, 0.00742, 
                    0.004151, 0.007117, 0.009745, 0.02452, 0.037877, 0.09556, 
                    0.205067, 0.282588, 0.234866, 0.184225, 0.132268, 0.101679]

    # Initially the horizon is the full forecast
    horizon = N
    first = 0
    problem_solved = False
    max_storage = 40
    operated_hours = []
    storage_capacity = [0]*horizon
    initial_storage = max_storage
    
    while not problem_solved:
        
        # Print horizon
        print("\n---------------------------------------")
        print(f"Looking into {first}:00-{horizon}:00.")
        print("---------------------------------------")
        
        # ---------------------------------------
        # Find cheapest operating hours that would provide the needed heat
        # ---------------------------------------
    
        # Get the heating required in the horizon
        needed_heating = sum(load[first:horizon])
        print(f"\nNeeded heating: {round(needed_heating,2)}")
            
        # Sort the hours by price
        hours_sorted = [cost_th_list.index(x) for x in sorted(cost_th_list[first:horizon])]
    
        # Assign operating hours until load is met
        provided_heating = 0
        operating_hours = []
        for hour in hours_sorted:
            operating_hours.append(hour)
            provided_heating += heating_capacity[hour]
            if provided_heating >= needed_heating:
                break
        #print(f"\nOperating hours:\n{operating_hours}")
    
        # ---------------------------------------
        # Get periods
        # ---------------------------------------
        
        periods = []
        current_sublist = []
        for num in sorted(operating_hours):
            if not current_sublist or num == current_sublist[-1] + 1:
                current_sublist.append(num)
            else:
                periods.append(current_sublist)
                current_sublist = [num]
    
        if current_sublist:
            periods.append(current_sublist)
    
        print(f"Operating periods:\n{periods}")
        
        # ---------------------------------------
        # See what this would imply for the storage
        # ---------------------------------------
    
        if storage_capacity[0] == 0:
            storage_capacity[0] = initial_storage - load[0]
            storage_capacity[0] += heating_capacity[0] if 0 in operating_hours else 0
              
        for k in range(first+1,horizon):
            # for every period
            for period in periods:
                if k in period:
                    heating = heating_capacity[k]
                    break
                else:
                    heating = 0
            storage_capacity[k] = storage_capacity[k-1] - load[k] + heating
        print(f"\nStorage capacity:\n{[round(x,1) for x in storage_capacity[:horizon]]}")
    
        # ---------------------------------------
        # Cut the horizon where the storage capacity becomes 0
        # ---------------------------------------
    
        ok = [1 if x<0 else 0 for x in storage_capacity]
        
        if sum(ok)!=0: 
            horizon = ok.index(1) + first + 1
            print(f"\nStarting over but within {first}:00-{horizon}:00.")
        else:
            if horizon==24:
                print("\nProblem solved.")
                problem_solved = True
                operated_hours.extend(operating_hours[:horizon])
                break
            first = horizon
            horizon = N
            operated_hours.extend(operating_hours[:first])
            print(f"\nWorked until {first}:00, now look at {first}:00-{horizon}:00.")
    
    # ---------------------------------------
    # Plot
    # ---------------------------------------
    
    # Get the heating over the 24 hours
    Q_HP = [heating_capacity[k] if k in operated_hours else 0 for k in range(N)]
    
    # Get the storage state over the 24 hours
    storage = [initial_storage] + [0]*(N)
    for i in range(N):
        storage[i+1] = storage[i] + Q_HP[i] - load[i]
    
    # Duplicate the last element of the hourly data for the plot
    cost_th_list2 = cost_th_list + [cost_th_list[-1]]
    load2 = load + [load[-1]]
    Q_HP = [round(x,3) for x in Q_HP + [Q_HP[-1]]]
        
    if problem_solved:
        
        # Plot
        fig, ax = plt.subplots(1,1, figsize=(13,4))
        ax.plot(range(N+1), storage, label='Storage', alpha=1, color='orange')
        ax.step(range(N+1), Q_HP, where='post', label='Q_HP', alpha=0.6, color='blue')
        ax.step(range(N+1), load2, where='post', label='Load', alpha=0.4, color='red')
        ax.set_xticks(range(N+1))
        ax.set_ylabel("Energy [$kWh_{th}$]")
        ax2 = ax.twinx()
        ax2.set_ylabel("Price [$/kWh_th]")
        ax2.step(range(N+1), cost_th_list2, where='post', label='Electricity price', color='black', alpha = 0.4)
        lines1, labels1 = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2)
        plt.show()

In [ ]:
peter_algo()